In [1]:
import json
import periodictable as pt

In [2]:
lipids = {}

class Lipid(object):
    def __init__(self, name, head_formula, tail_formula, head_exchangable=0,
                 tail_exchangable=0, structure=None, references=None,
                 conditions=None):
        self.name = name
        self.head_formula = head_formula
        self.tail_formula = tail_formula
        self.structure = structure
        self.references = references
        self.conditions = conditions
        if conditions is None:
            self.conditions = {}
        self.head_exchangable = head_exchangable
        self.tail_exchangable = tail_exchangable

    def __repr__(self):
        s = ("Lipid({name!r}, {head_formula!r}, {tail_formula!r},"
             " head_exchangable={head_exchangable!r},"
             " tail_exchangable={tail_exchangable!r},"
             " structure={structure!r}, references={references!r},"
             " conditions={conditions!r}")
        return s.format(**self.__dict__)

    def add_condition(self, descriptor, vh, vt):
        self.conditions[descriptor] = (vh, vt)

    @property
    def formula(self):
        return pt.formula(self.head_formula) + pt.formula(self.tail_formula)
    
    @property
    def tf(self):
        return pt.formula(self.tail_formula)

    @property
    def hf(self):
        return pt.formula(self.head_formula)

    def neutron_scattering_lengths(self, condition, vf_d_solvent=1,
                                   neutron_wavelength=1.8):
        vh, vt = self.conditions[condition]
        
        hf = exchange_protons_formula(self.hf,
                                      self.head_exchangable,
                                      vf_d_solvent)
        tf = exchange_protons_formula(self.tf,
                                      self.tail_exchangable,
                                      vf_d_solvent)

        h_density = calculate_density(hf, vh)
        t_density = calculate_density(tf, vt)

        h_sld = pt.neutron_sld(hf, density=h_density, wavelength=neutron_wavelength)
        t_sld = pt.neutron_sld(tf, density=t_density, wavelength=neutron_wavelength)
        h_scatlen = complex(*h_sld[0:2]) * vh / 1e6
        t_scatlen = complex(*t_sld[0:2]) * vt / 1e6
        return h_scatlen, t_scatlen

    def xray_scattering_lengths(self, condition, energy=8.048):
        vh, vt = self.conditions[condition]
        h_density = calculate_density(self.hf, vh)
        t_density = calculate_density(self.tf, vt)

        h_sld = pt.xray_sld(self.hf, density=h_density, energy=energy)
        t_sld = pt.xray_sld(self.tf, density=t_density, energy=energy)
        h_scatlen = complex(*h_sld[0:2]) * vh / 1e6
        t_scatlen = complex(*t_sld[0:2]) * vt / 1e6
        return h_scatlen, t_scatlen


def calculate_density(formula, volume):
    return formula.molecular_mass / formula.volume(a=volume, b=1, c=1)


def convert_atoms_to_formula(atoms):
    s = []
    for k, v in atoms.items():
        s.append([v, k])
    return pt.formula(s)


def exchange_protons_formula(formula, exchangable, vf_d):
    if not exchangable or pt.H not in formula.atoms:
        return formula

    atoms = formula.atoms
    initial = atoms[pt.H]
    atoms[pt.H] = initial - exchangable * vf_d
    atoms[pt.D] = exchangable * vf_d
    return convert_atoms_to_formula(atoms)

In [3]:
name = 'h-DMPC'
l = Lipid(name, 'C10H18O8NP', 'C26H54', structure=name + 'png',
          references=['DMPC in the fluid phase: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'DMPC in the gel phase: S. Tristram-Nagle et al. (2002) Biophys. J. 83: 3324-3335'])
l.add_condition('fluid30C', 319, 782)
l.add_condition('gel10C', 319, 710)
lipids[name] = l

In [4]:
name = 'd54-DMPC'
l = Lipid(name, 'C10H18O8NP', 'C26D54', structure=name + '.png',
              references=['DMPC in the fluid phase: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                          'DMPC in the gel phase: S. Tristram-Nagle et al. (2002) Biophys. J. 83: 3324-3335'])
l.add_condition('fluid30C', 319, 782)
l.add_condition('gel10C', 319, 710)
lipids[name] = l

In [5]:
name = 'h-DMPG'
l = Lipid(name, 'C8H12O10P1', 'C26H54', structure=name + '.png',
          head_exchangable=2,
          references=['Used fluid phase DMPC for tails volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'Used headgroup volume from DPPG: G. Pabst et al. (2007) Biophys. J. 93: 513-525'])
l.add_condition('ambient', 257, 782)
lipids[name] = l

In [6]:
name = 'd54-DMPG'
l = Lipid(name, 'C8H12O10P1', 'C26D54', structure=name + '.png',
          head_exchangable=2,
          references=['Used fluid phase DMPC for tails volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'Used headgroup volume from DPPG: G. Pabst et al. (2007) Biophys. J. 93: 513-525'])
l.add_condition('ambient', 257, 782)
lipids[name] = l

In [7]:
name = 'h-POPC'
l = Lipid(name, 'C10H18O8NP', 'C32H64', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [8]:
name = 'd31-POPC'
l = Lipid(name, 'C10H18O8NP', 'C32H33D31', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [9]:
name = 'd9-POPC'
l = Lipid(name, 'C10H9D9O8NP', 'C32H64', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [10]:
name = 'h-POPG'
l = Lipid(name, 'C8H12O10P', 'C32H64', structure=name + '.png',
          head_exchangable=2,
          references=['For obtaining heads volume: G. Pabst et al. (2007) Biophys. J. 93: 513-525',
                      'For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 257, 944)
lipids[name] = l

In [11]:
name = 'd31-POPG'
l = Lipid(name, 'C8H12O10P', 'C32H33D31', structure=name + '.png',
          head_exchangable=2,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: G. Pabst et al. (2007) Biophys. J. 93: 513-525 '])
l.add_condition('ambient', 257, 944)
lipids[name] = l

In [12]:
name = 'h-POPS'
l = Lipid(name, 'C8H11O10NP', 'C32H64', structure=name + '.png',
          head_exchangable=1,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: H.I. Petrache et al. (2004) Biophys. J. 86: 1574-1586'])
l.add_condition('ambient', 244, 944)
lipids[name] = l

In [13]:
name = 'd31-POPS'
l = Lipid(name, 'C8H11O10NP', 'C32H33D31', structure=name + '.png',
          head_exchangable=1,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: H.I. Petrache et al. (2004) Biophys. J. 86: 1574-1586'])
l.add_condition('ambient', 244, 944)
lipids[name] = l

In [14]:
name = 'h-POPE'
l = Lipid(name, 'C7H12O8P', 'C32H64', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [15]:
name = 'd31-POPE'
l = Lipid(name, 'C7H12O8P', 'C32H33D31', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [16]:
name = 'd4-POPE'
l = Lipid(name, 'C7H8D4O8P', 'C32H64', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [17]:
name = 'h-DPPC'
l = Lipid(name, 'C10H18O8NP', 'C30H62', structure=name + '.png',
          head_exchangable=0,
          references=['J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('gel20C', 319, 825)
l.add_condition('fluid50C', 319, 913)
lipids[name] = l

In [18]:
name = 'd62-DPPC'
l = Lipid(name, 'C10H18O8NP', 'C30D62', structure=name + '.png',
          head_exchangable=0,
          references=['J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('gel20C', 319, 825)
l.add_condition('fluid50C', 319, 913)
lipids[name] = l

In [20]:
with open('lipids.json', 'w') as f:
    json.dump([lipids[p].__dict__ for p in lipids], f)